In [1]:
from datetime import date
import ast
import pandas as pd

pd.set_option('display.max_columns', None)
df = catalog.load('raw/openaire/researchproduct_collectedfrom_datasource#parquet')

                    INFO     Loading data from                                                  ]8;id=335102;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=332933;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             raw/openaire/researchproduct_collectedfrom_datasource#parquet                         
                             (ParquetDataset)...                                                                   

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 0 to 49
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author              300 non-null    object
 1   openAccessColor     47 non-null     object
 2   publiclyFunded      283 non-null    object
 3   type                300 non-null    object
 4   language            300 non-null    object
 5   country             300 non-null    object
 6   subjects            300 non-null    object
 7   mainTitle           300 non-null    object
 8   description         300 non-null    object
 9   publicationDate     300 non-null    object
 10  format              287 non-null    object
 11  contributor         35 non-null     object
 12  bestAccessRight     213 non-null    object
 13  id                  300 non-null    object
 14  originalId          300 non-null    object
 15  indicators          300 non-null    object
 16  instance            300 non-null

In [3]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'country', 'subjects', 'mainTitle', 'description', 'publicationDate',
       'format', 'contributor', 'bestAccessRight', 'id', 'originalId',
       'indicators', 'instance', 'isGreen', 'isInDiamondJournal', 'publisher',
       'source', 'container', 'pid', 'contactPerson'],
      dtype='object')

In [4]:
df_researchproduct2instance = df.explode('instance').reset_index(drop=True)
df_researchproduct2instance = df_researchproduct2instance[['id','instance']]
df_instance = pd.json_normalize(df_researchproduct2instance['instance']).reset_index(drop=True)
df_researchproduct2instance = pd.concat([df_researchproduct2instance.drop(columns=['instance']), df_instance], axis=1)
df_researchproduct2url = df_researchproduct2instance[['id','url']]
df_researchproduct2url = df_researchproduct2url.explode('url')

In [5]:
pd.json_normalize(df['instance'].explode())['url'].explode()


0          http://sedici.unlp.edu.ar/handle/10915/154923
1                                                   None
2      https://doaj.org/article/a86e0a7dd7ad4000ae40a...
3           http://sedici.unlp.edu.ar/handle/10915/97519
4           http://sedici.unlp.edu.ar/handle/10915/46429
                             ...                        
677         http://sedici.unlp.edu.ar/handle/10915/10585
678                                                 None
679                                                 None
680         http://sedici.unlp.edu.ar/handle/10915/66426
681        http://sedici.unlp.edu.ar/handle/10915/146888
Name: url, Length: 706, dtype: object

## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [6]:
def land_openaire_researchproduct(df: pd.DataFrame)-> pd.DataFrame:

    df = df.convert_dtypes()

    expected_columns = [
        'author',
        'openAccessColor',
        'publiclyFunded',
        'type',
        'language',
        'country',
        'subjects',
        'mainTitle',
        'description',
        'publicationDate',
        'format',
        'bestAccessRight',
        'id',
        'originalId',
        'indicators',
        'instance',
        'isGreen',
        'isInDiamondJournal',
        'publisher',
        'source',
        'container',
        'contributor',
        'contactPerson',
        'coverage',
        'pid',
        'contactPerson',
        'embargoEndDate'
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df_researchproduct = df[expected_columns].copy()
    df.reset_index(drop=True, inplace=True)

    # language
    df_researchproduct['language_code'] = df_researchproduct['language'].apply(lambda x: x['code'])
    df_researchproduct['language_label'] = df_researchproduct['language'].apply(lambda x: x['label'])

    ## bestAccessRight
    df_researchproduct['bestAccessRight_label'] = df['bestAccessRight'].apply(lambda x: x['label'] if x else None)
    df_researchproduct['bestAccessRight_scheme'] = df['bestAccessRight'].apply(lambda x: x['scheme'] if x else None)

    ## indicators
    df_indicators = pd.json_normalize(df['indicators']).reset_index(drop=True)
    df_researchproduct = pd.concat([df_researchproduct.drop(columns=['indicators']).reset_index(drop=True), df_indicators], axis=1)

    ## author
    df_researchproduct2author = df.explode('author').reset_index(drop=True)
    df_researchproduct2author = df_researchproduct2author[['id','author']]
    df_authors = pd.json_normalize(df_researchproduct2author['author']).reset_index(drop=True)
    df_researchproduct2author = pd.concat([df_researchproduct2author.drop(columns=['author']), df_authors], axis=1)

    ## originalId
    df_researchproduct2originalId = df.explode('originalId').reset_index(drop=True)
    df_researchproduct2originalId = df_researchproduct2originalId[['id','originalId']]

    # TODO country

    ## subjects
    df_researchproduct2subject = df.explode('subjects').reset_index(drop=True)
    df_researchproduct2subject = df_researchproduct2subject[['id','subjects']]
    df_subjects = pd.json_normalize(df_researchproduct2subject['subjects']).reset_index(drop=True)
    df_researchproduct2subject = pd.concat([df_researchproduct2subject.drop(columns=['subjects']), df_subjects], axis=1)

    # TODO description

    # TODO format

    # TODO instance

    # TODO source

    # TODO container
    
    # TODO contributor
    
    # TODO contactPerson

    # TODO coverage
    
    # pid
    df_researchproduct2pid = df.explode('pid').reset_index(drop=True)
    df_researchproduct2pid = df_researchproduct2pid[['id','pid']]
    df_pid = pd.json_normalize(df_researchproduct2pid['pid']).reset_index(drop=True)
    df_researchproduct2pid = pd.concat([df_researchproduct2pid.drop(columns=['pid']), df_pid], axis=1)

    # url
    df_researchproduct2instance = df.explode('instance').reset_index(drop=True)
    df_researchproduct2instance = df_researchproduct2instance[['id','instance']]
    df_instance = pd.json_normalize(df_researchproduct2instance['instance']).reset_index(drop=True)
    df_researchproduct2instance = pd.concat([df_researchproduct2instance.drop(columns=['instance']), df_instance], axis=1)
    df_researchproduct2url = df_researchproduct2instance[['id','url']]
    df_researchproduct2url = df_researchproduct2url.explode('url')

    ## drop de columnas procesadas en otros df
    df_researchproduct.drop(columns=[
        'author', 'country', 'subjects','bestAccessRight', 
        'language', 'format', 'instance', 'originalId', 
        'container', 'source', 'pid', 'description',
        'contributor', 'contactPerson', 'coverage'
        ], inplace=True)

    df_researchproduct['load_datetime'] = date.today()
    df_researchproduct2originalId['load_datetime'] = date.today()
    df_researchproduct2author['load_datetime'] = date.today()
    df_researchproduct2subject['load_datetime'] = date.today()
    df_researchproduct2pid['load_datetime'] = date.today()
    df_researchproduct2url['load_datetime'] = date.today()

    return df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject, df_researchproduct2pid, df_researchproduct2url


In [7]:
df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject, df_researchproduct2pid, df_researchproduct2url = land_openaire_researchproduct(df)

In [8]:
df_researchproduct

,openAccessColor,publiclyFunded,type,mainTitle,publicationDate,id,isGreen,isInDiamondJournal,publisher,embargoEndDate,language_code,language_label,bestAccessRight_label,bestAccessRight_scheme,citationImpact.citationClass,citationImpact.citationCount,citationImpact.impulse,citationImpact.impulseClass,citationImpact.influence,citationImpact.influenceClass,citationImpact.popularity,citationImpact.popularityClass,load_datetime
0,<NA>,False,publication,Evolución del sistema de gestión de incidentes...,2023-06-23,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,True,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,2.885067e-09,C5,2025-02-07
1,gold,False,publication,Responsabilidad Social Corporativa en Empresas...,2013-12-01,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,False,False,Universidad Nacional de La Plata,<NA>,eng,English,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,7.596338e-10,C5,2025-02-07
2,<NA>,<NA>,other,Interfaces naturales como factor de progreso c...,2015-04-01,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,<NA>,<NA>,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,9.624046e-10,C5,2025-02-07
3,<NA>,False,publication,"Jornadas, exposiciones y encuesta de opinión",2010-01-01,dedup_wf_002::00027faeda057626e8abbaba310695fa,True,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,5.546645e-10,C5,2025-02-07
4,<NA>,False,publication,Sobre temporalidad y territorio,2012-08-01,dedup_wf_002::00043054b1c5eaa3ee50733d77a9be9a,True,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,6.801357e-10,C5,2025-02-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,<NA>,False,publication,Mejoramiento del diagnóstico de la tuberculosi...,2005-01-01,dedup_wf_002::0184680ec06446e3f27ca2d276b004b7,True,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,3.714550e-10,C5,2025-02-07
296,<NA>,False,publication,La complejidad del delito de trata : Aportes d...,2022-01-01,dedup_wf_002::018651398348d30c1bc30d65f3285651,False,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,2.491310e-09,C5,2025-02-07
297,gold,False,publication,"Traducción de poesía: forma, repetición y fant...",2005-11-01,dedup_wf_002::018dc4a99e0bcacf2b8012f79d497329,True,False,Universidad Nacional de La Plata,<NA>,esl/spa,Spanish,None,None,C5,0.0,0.0,C5,2.841867e-09,C5,3.714550e-10,C5,2025-02-07
298,<NA>,False,publication,Arte rupestre y demarcación territorial: el ca...,2018-04-01,dedup_wf_002::018e7a4d415d7a560694e107f3ddecfa,False,False,<NA>,<NA>,esl/spa,Spanish,None,None,C5,0.0,0.0,C5,2.841867e-09,C5,1.418572e-09,C5,2025-02-07


In [9]:
df_researchproduct2originalId.head(5)

,id,originalId,load_datetime
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,oai:sedici.unlp.edu.ar:10915/154923,2025-02-07
1,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,50|od______1329::f21670428c3312fd0facb5ae0211c97b,2025-02-07
2,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,ftunivlaplata:oai:sedici.unlp.edu.ar:10915/154923,2025-02-07
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,50|base_oa_____::4976a9c78617e968aa278af4f762dff1,2025-02-07
4,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,oai:doaj.org/article:a86e0a7dd7ad4000ae40ad078...,2025-02-07


In [10]:
df_researchproduct2author.head(5)

,id,fullName,name,pid,rank,surname,pid.id.scheme,pid.id.value,pid.provenance,load_datetime
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,"Rubio, Damián",Damián,NaN,1,Rubio,NaN,NaN,NaN,2025-02-07
1,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,"Ojeda Hidalgo, José F.",José F.,NaN,1,Ojeda Hidalgo,NaN,NaN,NaN,2025-02-07
2,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,"Conraud Koellner, Eva",Eva,NaN,2,Conraud Koellner,NaN,NaN,NaN,2025-02-07
3,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,"Archundia Fernández, Emigdio",Emigdio,NaN,3,Archundia Fernández,NaN,NaN,NaN,2025-02-07
4,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,"Contreras, Víctor",Víctor,NaN,1,Contreras,NaN,NaN,NaN,2025-02-07


In [11]:
df_researchproduct2subject

,id,provenance,subject.scheme,subject.value,load_datetime
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None,keyword,Gestión de incidentes,2025-02-07
1,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None,keyword,CSIRT,2025-02-07
2,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None,keyword,Ciencias Informáticas,2025-02-07
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None,keyword,ciberseguridad,2025-02-07
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None,keyword,CERT,2025-02-07
...,...,...,...,...,...
2275,dedup_wf_002::018e7a4d415d7a560694e107f3ddecfa,None,keyword,"arte rupestre, paisagens culturais, Patagônia ...",2025-02-07
2276,dedup_wf_002::018f15b2ff0ac650669c61fe70c4c8d9,None,keyword,adhesión,2025-02-07
2277,dedup_wf_002::018f15b2ff0ac650669c61fe70c4c8d9,None,keyword,perfil de densidad electrónica,2025-02-07
2278,dedup_wf_002::018f15b2ff0ac650669c61fe70c4c8d9,None,keyword,Oxígeno,2025-02-07


In [12]:
df_researchproduct2pid[df_researchproduct2pid['value'].notna()]

,id,scheme,value,load_datetime
5,dedup_wf_002::0005b301e367f0fc799b29dc387d3bd1,handle,11336/112574,2025-02-07
16,dedup_wf_002::0011bff3ad8c8b581b41ac3ea5935947,handle,11336/49652,2025-02-07
18,dedup_wf_002::00187ed7570bf3d9d39329e65003a02c,handle,11336/127690,2025-02-07
32,dedup_wf_002::00277ef4a8434a8b5f638f00ea875a0d,handle,11336/87684,2025-02-07
46,dedup_wf_002::00444c2ce8204cabf69a2bf42ce79211,handle,11336/22280,2025-02-07
49,dedup_wf_002::00483b1ae3d99ee85cc99f45189c6c82,mag_id,2776895693,2025-02-07
50,dedup_wf_002::00483b1ae3d99ee85cc99f45189c6c82,doi,10.30827/rl.v0i19.6670,2025-02-07
51,dedup_wf_002::00483b1ae3d99ee85cc99f45189c6c82,handle,10481/58952,2025-02-07
52,dedup_wf_002::00483b1ae3d99ee85cc99f45189c6c82,handle,11336/61474,2025-02-07
57,dedup_wf_002::0050375ef399f4bf27533b4638d61b8b,handle,11336/166916,2025-02-07


In [13]:
df_researchproduct2url

,id,url
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,http://sedici.unlp.edu.ar/handle/10915/154923
1,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None
2,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,https://doaj.org/article/a86e0a7dd7ad4000ae40a...
3,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,http://sedici.unlp.edu.ar/handle/10915/97519
4,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,http://sedici.unlp.edu.ar/handle/10915/46429
...,...,...
677,dedup_wf_002::018dc4a99e0bcacf2b8012f79d497329,http://sedici.unlp.edu.ar/handle/10915/10585
678,dedup_wf_002::018dc4a99e0bcacf2b8012f79d497329,None
679,dedup_wf_002::018dc4a99e0bcacf2b8012f79d497329,None
680,dedup_wf_002::018e7a4d415d7a560694e107f3ddecfa,http://sedici.unlp.edu.ar/handle/10915/66426
